In [1]:
import numpy as np

## Definition of elastic tensor with $\nu > 0.25$

In [2]:
E = 1.
nu = 0.4

la = E * nu / ((1. + nu) * (1. - 2. * nu))
mu = E / (2. + 2. * nu)

delta = np.identity(3)

D_abef = (np.einsum(',ij,kl->ijkl', la, delta, delta) +
          np.einsum(',ik,jl->ijkl', mu, delta, delta) +
          np.einsum(',il,jk->ijkl', mu, delta, delta))

## Definition of strain and stress tensor, uniaxial compression

In [3]:
eps = np.array([[-1, 0, 0],[0, 0.4, 0],[0, 0, 0.4]])

sigma = np.einsum('ijkl,kl->ij',D_abef,eps)

print('\epsilon =' ,eps)
print('\sigma =' ,sigma)

\epsilon = [[-1.   0.   0. ]
 [ 0.   0.4  0. ]
 [ 0.   0.   0.4]]
\sigma = [[-1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -1.11022302e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.11022302e-16]]


## Kinematic constraint

\begin{align}
\varepsilon_{\mathrm{N}} &= \varepsilon_{ij} n_i n_j\\
\nonumber
\varepsilon_{\mathrm{T}_r} &= \frac{1}{2}
\left(n_i \delta_{jr} + n_j \delta_{ir} - 2 n_i n_j n_r\right) \,  \varepsilon_{ij}
\end{align}

In [4]:
n = np.array([.250562787, -.935113132, .250562787])

eps_N = np.einsum('ij,i,j->',eps,n,n)
eps_T = 0.5 * np.einsum('i,jr,ij->r',n,delta,eps) + 0.5 * np.einsum('j,ir,ij->r',n,delta,eps)\
        - np.einsum('i,j,r,ij->r',n,n,n,eps)
print('\epsilon_N = ',eps_N)
print('\epsilon_T = ',eps_T)

\epsilon_N =  0.3121056017180153
\epsilon_T =  [-0.32876484 -0.08219121  0.02202307]


## Static constraint

\begin{align}
\sigma_{\mathrm{N}} &= \sigma_{ij} n_i n_j\\
\nonumber
\sigma_{\mathrm{T}_r} &= \frac{1}{2}
\left(n_i \delta_{jr} + n_j \delta_{ir} - 2 n_i n_j n_r\right) \,  \sigma_{ij}
\end{align}

In [5]:
sigma_N_static = np.einsum('ij,i,j->',sigma,n,n)
sigma_T_statin = 0.5 * np.einsum('i,jr,ij->r',n,delta,sigma) + 0.5 * np.einsum('j,ir,ij->r',n,delta,sigma)\
        - np.einsum('i,j,r,ij->r',n,n,n,sigma)
print('\sigma_N_static =' ,sigma_N_static)
print('\sigma_T_static =',sigma_T_statin)

\sigma_N_static = -0.06278171022920749
\sigma_T_static = [-0.23483203 -0.058708    0.01573076]


## Applying microplane material law

In [6]:
E_N = E / (1.0 - 2.0 * nu)
E_T = (E * (1.0 - 4.0 * nu)) / ((1.0 + nu) * (1.0 - 2.0 * nu))

sigma_N = E_N * eps_N
sigma_T = E_T * eps_T

print('\sigma_N_micro =' ,sigma_N)
print('\sigma_T_micro =' ,sigma_T)

\sigma_N_micro = 1.5605280085900768
\sigma_T_micro = [ 0.70449608  0.17612401 -0.04719228]


## We are getting microplane stress vector going in the opposite direction of strain vector 
## What happens if we integrate the microplane stresses?

From PVW or integrating microplane free energy, same result. By: Carol, Bazant, Jirasek

\begin{equation}
\sigma_{ij} = \frac{3}{4\pi} \int_{\Omega} \sigma_{\mathrm{N}} n_i n_j d\Omega + \frac{3}{4\pi} \int_{\Omega} \frac{\sigma_{\mathrm{T}_r}}{2} (n_i \delta_{rj} + n_j \delta_{ri}) d\Omega \hspace{1cm}
\label{eq:sigma_ij}
\end{equation}

## 28 microplanes hemisphere, weights factors and normals, Bazant

In [7]:
MPW = np.array([.0160714276, .0160714276, .0160714276, .0160714276, .0204744730,
                      .0204744730, .0204744730, .0204744730, .0204744730, .0204744730,
                      .0204744730, .0204744730, .0204744730, .0204744730, .0204744730,
                      .0204744730, .0158350505, .0158350505, .0158350505, .0158350505,
                      .0158350505, .0158350505, .0158350505, .0158350505, .0158350505,
                      .0158350505, .0158350505, .0158350505]) * 6.0

MPN = np.array([[.577350259, .577350259, .577350259],
                      [.577350259, .577350259, -.577350259],
                      [.577350259, -.577350259, .577350259],
                      [.577350259, -.577350259, -.577350259],
                      [.935113132, .250562787, .250562787],
                      [.935113132, .250562787, -.250562787],
                      [.935113132, -.250562787, .250562787],
                      [.935113132, -.250562787, -.250562787],
                      [.250562787, .935113132, .250562787],
                      [.250562787, .935113132, -.250562787],
                      [.250562787, -.935113132, .250562787],
                      [.250562787, -.935113132, -.250562787],
                      [.250562787, .250562787, .935113132],
                      [.250562787, .250562787, -.935113132],
                      [.250562787, -.250562787, .935113132],
                      [.250562787, -.250562787, -.935113132],
                      [.186156720, .694746614, .694746614],
                      [.186156720, .694746614, -.694746614],
                      [.186156720, -.694746614, .694746614],
                      [.186156720, -.694746614, -.694746614],
                      [.694746614, .186156720, .694746614],
                      [.694746614, .186156720, -.694746614],
                      [.694746614, -.186156720, .694746614],
                      [.694746614, -.186156720, -.694746614],
                      [.694746614, .694746614, .186156720],
                      [.694746614, .694746614, -.186156720],
                      [.694746614, -.694746614, .186156720],
                      [.694746614, -.694746614, -.186156720]])

## 2 order N tensor, 3rd order T tensor

In [8]:
MPTT_ijr = 0.5 * (np.einsum('ni,jr -> nijr', MPN, delta) +
                  np.einsum('nj,ir -> njir', MPN, delta) - 2 *
                  np.einsum('ni,nj,nr -> nijr', MPN, MPN, MPN))

MPNN = np.einsum('ni,nj->nij', MPN, MPN)



## Applying kinematic constraint

In [9]:
eps_N_Emn = np.einsum('nij,...ij->...n', MPNN, eps) 

eps_T_Emna = np.einsum('nija,...ij->...na', MPTT_ijr, eps)

print('\epsilon_N = ',eps_N_Emn[10])
print('\epsilon_T = ',eps_T_Emna[10])

\epsilon_N =  0.3121056017180154
\epsilon_T =  [-0.32876484 -0.08219121  0.02202307]


## Microplane material law

In [10]:
sigma_N_Emn = E_N * eps_N_Emn
sigma_T_Emna = E_T * eps_T_Emna

print('\sigma_N_micro =' ,sigma_N_Emn[10])
print('\sigma_T_micro =' ,sigma_T_Emna[10])

\sigma_N_micro = 1.5605280085900772
\sigma_T_micro = [ 0.70449608  0.17612401 -0.04719228]


## Numerical integration of microplane stresses

In [11]:
sigma_int = np.einsum('n,...n,na,nb->...ab', MPW, sigma_N_Emn, MPN, MPN) + \
            0.5 * (np.einsum('n,...nf,na,fb->...ab', MPW, sigma_T_Emna, MPN, delta) +
                   np.einsum('n,...nf,nb,fa->...ab', MPW, sigma_T_Emna, MPN, delta))

print('integrated \sigma = ',sigma_int)
print('original \sigma = ',sigma)

integrated \sigma =  [[-1.00000026e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.31586186e-07  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.31586186e-07]]
original \sigma =  [[-1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -1.11022302e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.11022302e-16]]


\begin{align}
\varepsilon_{\mathrm{N}} &= \varepsilon_{ij} n_i n_j\\
\nonumber
\varepsilon_{\mathrm{T}_r} &= \frac{1}{2}
\left(n_i \delta_{jr} + n_j \delta_{ir} - 2 n_i n_j n_r\right) \,  \varepsilon_{ij}
\end{align}

In [12]:
n = np.array([.250562787, -.935113132, .250562787])

eps_N = np.einsum('ij,i,j->',eps,n,n)
eps_T = 0.5 * np.einsum('i,jr,ij->r',n,delta,eps) + 0.5 * np.einsum('j,ir,ij->r',n,delta,eps)\
        - np.einsum('i,j,r,ij->r',n,n,n,eps)
print('\epsilon_N = ',eps_N)
print('\epsilon_T = ',eps_T)

\epsilon_N =  0.3121056017180153
\epsilon_T =  [-0.32876484 -0.08219121  0.02202307]


## Static constraint

\begin{align}
\sigma_{\mathrm{N}} &= \sigma_{ij} n_i n_j\\
\nonumber
\sigma_{\mathrm{T}_r} &= \frac{1}{2}
\left(n_i \delta_{jr} + n_j \delta_{ir} - 2 n_i n_j n_r\right) \,  \sigma_{ij}
\end{align}

In [22]:
sigma_N_static = np.einsum('ij,i,j->',sigma,n,n)
sigma_T_statin = 0.5 * np.einsum('i,jr,ij->r',n,delta,sigma) + 0.5 * np.einsum('j,ir,ij->r',n,delta,sigma)\
        - np.einsum('i,j,r,ij->r',n,n,n,sigma)
print('\sigma_N_static =' ,sigma_N_static)
print('\sigma_T_static =',sigma_T_statin)



\sigma_N_static = -0.06278171022920749
\sigma_T_static = [-0.23483203 -0.058708    0.01573076]


## Applying microplane material law

In [23]:
E_N = E / (1.0 - 2.0 * nu)
E_T = (E * (1.0 - 4.0 * nu)) / ((1.0 + nu) * (1.0 - 2.0 * nu))

sigma_N = E_N * eps_N
sigma_T = E_T * eps_T

print('\sigma_N_micro =' ,sigma_N)
print('\sigma_T_micro =' ,sigma_T)

\sigma_N_micro = 1.5605280085900768
\sigma_T_micro = [ 0.70449608  0.17612401 -0.04719228]


## We are getting microplane stress vector going in the opposite direction of strain vector 
## What happens if we integrate the microplane stresses?

From PVW or integrating microplane free energy, same result. By: Carol, Bazant, Jirasek

\begin{equation}
\sigma_{ij} = \frac{3}{4\pi} \int_{\Omega} \sigma_{\mathrm{N}} n_i n_j d\Omega 
 + \frac{3}{4\pi} \int_{\Omega} \frac{\sigma_{\mathrm{T}_r}}{2} (n_i \delta_{rj} + n_j \delta_{ri}) d\Omega \hspace{1cm}
\label{eq:sigma_ij}
\end{equation}

## 28 microplanes hemisphere, weights factors and normals, Bazant

In [24]:
MPW = np.array([.0160714276, .0160714276, .0160714276, .0160714276, .0204744730,
                      .0204744730, .0204744730, .0204744730, .0204744730, .0204744730,
                      .0204744730, .0204744730, .0204744730, .0204744730, .0204744730,
                      .0204744730, .0158350505, .0158350505, .0158350505, .0158350505,
                      .0158350505, .0158350505, .0158350505, .0158350505, .0158350505,
                      .0158350505, .0158350505, .0158350505]) * 6.0

MPN = np.array([[.577350259, .577350259, .577350259],
                      [.577350259, .577350259, -.577350259],
                      [.577350259, -.577350259, .577350259],
                      [.577350259, -.577350259, -.577350259],
                      [.935113132, .250562787, .250562787],
                      [.935113132, .250562787, -.250562787],
                      [.935113132, -.250562787, .250562787],
                      [.935113132, -.250562787, -.250562787],
                      [.250562787, .935113132, .250562787],
                      [.250562787, .935113132, -.250562787],
                      [.250562787, -.935113132, .250562787],
                      [.250562787, -.935113132, -.250562787],
                      [.250562787, .250562787, .935113132],
                      [.250562787, .250562787, -.935113132],
                      [.250562787, -.250562787, .935113132],
                      [.250562787, -.250562787, -.935113132],
                      [.186156720, .694746614, .694746614],
                      [.186156720, .694746614, -.694746614],
                      [.186156720, -.694746614, .694746614],
                      [.186156720, -.694746614, -.694746614],
                      [.694746614, .186156720, .694746614],
                      [.694746614, .186156720, -.694746614],
                      [.694746614, -.186156720, .694746614],
                      [.694746614, -.186156720, -.694746614],
                      [.694746614, .694746614, .186156720],
                      [.694746614, .694746614, -.186156720],
                      [.694746614, -.694746614, .186156720],
                      [.694746614, -.694746614, -.186156720]])

## 2 order N tensor, 3rd order T tensor

In [25]:
MPTT_ijr = 0.5 * (np.einsum('ni,jr -> nijr', MPN, delta) +
                   np.einsum('nj,ir -> njir', MPN, delta) - 2 *
                   np.einsum('ni,nj,nr -> nijr', MPN, MPN, MPN))

MPNN = np.einsum('ni,nj->nij', MPN, MPN)

## Applying kinematic constraint

In [26]:
eps_N_Emn = np.einsum('nij,...ij->...n', MPNN, eps) 
eps_T_Emna = np.einsum('nija,...ij->...na', MPTT_ijr, eps)

print('\epsilon_N = ',eps_N_Emn[10])
print('\epsilon_T = ',eps_T_Emna[10])

\epsilon_N =  0.3121056017180154
\epsilon_T =  [-0.32876484 -0.08219121  0.02202307]


## Microplane material law

In [27]:
sigma_N_Emn = E_N * eps_N_Emn
sigma_T_Emna = E_T * eps_T_Emna

print('\sigma_N_micro =' ,sigma_N_Emn[10])
print('\sigma_T_micro =' ,sigma_T_Emna[10])

\sigma_N_micro = 1.5605280085900772
\sigma_T_micro = [ 0.70449608  0.17612401 -0.04719228]


## Numerical integration of microplane stresses

In [28]:
sigma_int = np.einsum('n,...n,na,nb->...ab', MPW, sigma_N_Emn, MPN, MPN) + \
            0.5 * (np.einsum('n,...nf,na,fb->...ab', MPW, sigma_T_Emna, MPN, delta) +
                   np.einsum('n,...nf,nb,fa->...ab', MPW, sigma_T_Emna, MPN, delta))

print('integrated \sigma = ',sigma_int)
print('original \sigma = ',sigma)

integrated \sigma =  [[-1.00000026e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.31586186e-07  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.31586186e-07]]
original \sigma =  [[-1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -1.11022302e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.11022302e-16]]


## Analyze the Lame parameters

In [29]:
import sympy as sp

In [30]:
E_, nu_, lambda_, mu_ = sp.symbols('E, nu, lambda, mu')

In [31]:
eq_lambda = sp.Eq(lambda_, E_*nu_/ ((1+nu_)*(1-2*nu_)))

In [32]:
eq_mu = sp.Eq(mu_, E_ / (2*(1+nu_)))

In [33]:
eq_lambda

Eq(lambda, E*nu/((1 - 2*nu)*(nu + 1)))

In [34]:
sp.solve((E_*nu_/ ((1+nu_)*(1-2*nu_))).diff(nu_),nu_)

[-sqrt(2)*I/2, sqrt(2)*I/2]

In [35]:
eq_mu

Eq(mu, E/(2*nu + 2))

## Verification of macroscopic plastic strain tensor recovery

The decomposition of the elastic and inelastic terms in the homogenization procedure of the type
\begin{align}
\boldsymbol{\sigma} = \boldsymbol{\beta}:\boldsymbol{C}^{\mathrm{e}}:\boldsymbol{\beta}^{\mathrm{T}}:(\boldsymbol{\varepsilon} - \boldsymbol{\varepsilon}^\mathrm{p}). 
\end{align}

poses the question, if the homogenization of plastic microplane strains $\varepsilon_N^\mathrm{p,mic}$ and $\varepsilon_{Tr}^{\mathrm{p,mic}}$ correctly reproduces the macroscopic plastic strain $\varepsilon^p_{ij}$. The particular question is, if the resulting tensor preserves all symmetries. The motivation of this question is related to the MDM [Jirasek] which does not implicitly render  symmetric tensor and requires a subsequent symmetrization.

The hemisphere integration of microplane strain contributions is given as 

\begin{align} \label{eq:plastic_strain_tensor} 
\varepsilon^{p}_{ij} &= \frac{3}{2 \pi} \int_{\Omega}  \varepsilon^{p,\mathrm{mic}}_\mathrm{N}  n_i n_j  \, d \Omega + \frac{3}{2 \pi} \int_{\Omega} \frac{\varepsilon^{\mathrm{p,mic}}_{\mathrm{T}r}}{2} (n_i  \delta_{rj} + n_j \delta_{ri}) \, d \Omega.
\end{align}

The kinematic constraint is used to obtain the microplane strain from the tensor representation.

\begin{align}
\label{eq:epsilon_NT}
\varepsilon_{\mathrm{N}}^{p,\mathrm{mic}} &= \varepsilon_{ij}^{p} n_i n_j, \\
\varepsilon_{\mathrm{T}_r}^{p,\mathrm{mic}} &= \frac{1}{2}
\left(n_i \delta_{jr} + n_j \delta_{ir} - 2 n_i n_j n_r\right)  \varepsilon_{ij}^p. \nonumber
\end{align}

Substituting the projections to the integral we get

\begin{align} \label{eq:plastic_strain_tensor2} 
\varepsilon_{ij}^p &= 
\varepsilon_{kl}^p
\left[
\frac{3}{2 \pi} \int_{\Omega}  n_i n_j n_k n_l  \, d \Omega 
+ 
\frac{3}{8 \pi} \int_{\Omega} 
\left(n_i \delta_{jr} + n_j \delta_{ir} - 2 n_i n_j n_r\right) (n_i  \delta_{rj} + n_j \delta_{ri}) \, d \Omega.
\right]
\end{align}

After rearranging terms, the equivalence is obtained

\begin{align} \label{eq:plastic_strain_tensor3} 
\varepsilon_{ij}^p &= 
\varepsilon_{kl}^p
\left[
\frac{1}{5} 
\left(\delta_{ij}\delta_{kl} + \delta_{ik}\delta_{jl} + \delta_{il} \delta_{jk}
\right)
+ 
\left(\frac{3}{10} \delta_{ik} \delta_{jl} + \frac{3}{10} \delta_{il} \delta_{jk} - \frac{1}{5} \delta_{ij}\delta_{kl}
\right)\right] \\ \nonumber
&= 
\frac{1}{2}\left[ \delta_{ik} \delta_{jl} +  \delta_{il} \delta_{jk}
\right]
\varepsilon_{kl}^p
\end{align}


Note that the obtained term represents a symmetrization operator which has been explicitly used by Jirasek for microplane damage model. In his version, only normal strains were considered $\varepsilon_N$ and the symmetrization has been added as a correction of the lack of symmetry involved in the homogenization procedure. If the full decomposition into $\varepsilon_N$ and $\varepsilon_{Tr}$ is employed, a symmetric tensor is obtained implicitly.

## Sympy tensor package

Does the package provide a framework for symbolic verification of the above concepts, or even a code generator?

In [36]:
from sympy.tensor.tensor import TensorIndexType, TensorHead, TensorSymmetry
Lorentz = TensorIndexType('Lorentz', dummy_name='L')
asym2 = TensorSymmetry.fully_symmetric(-2)
A = TensorHead('A', [Lorentz, Lorentz], asym2)

In [37]:
A

A(Lorentz,Lorentz)